# Sesión 12: Redes Neuronales Recurrentes

---

Jueves 2 de Noviembre de 2023

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import os

### Importación de texto de entrenamiento

In [2]:
path_to_fileDL = tf.keras.utils.get_file('DL-Introduccion-practica-con-Keras-1a.txt', 'https://raw.githubusercontent.com/jorditorresBCN/Deep-Learning-Introduccion-practica-con-Keras/master/DeepLearning-Introduccion-practica-con-Keras-PRIMERA-PARTE.txt')

text = open(path_to_fileDL, 'rb').read().decode(encoding='utf-8')  
print ('Longitud del texto: {} caracteres'.format(len(text)))

vocab = sorted(set(text)) # vocabulario (todos los caracteres diferentes)
print ('Caracteres diferentes: {}'.format(len(vocab))) 

print('Vocabulario: ', vocab)

Longitud del texto: 203251 caracteres
Caracteres diferentes: 92
Vocabulario:  ['\n', '\r', ' ', '!', '"', '#', '%', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xad', 'ÿ', 'Š', '‡', '…']


### Tokenización e inversión del vocabulario

In [3]:
char2idx = {u:i for i, u in enumerate(vocab)} # diccionario que asigna un índice a cada caracter

idx2char = np.array(vocab) # array que asigna un caracter a cada índice (el inverso del anterior)

for char, _ in zip(char2idx, range(len(vocab))):
    print('{:6s} ---> {:4d}'.format(repr(char), char2idx[char]))

'\n'   --->    0
'\r'   --->    1
' '    --->    2
'!'    --->    3
'"'    --->    4
'#'    --->    5
'%'    --->    6
"'"    --->    7
'('    --->    8
')'    --->    9
'*'    --->   10
'+'    --->   11
','    --->   12
'-'    --->   13
'.'    --->   14
'/'    --->   15
'0'    --->   16
'1'    --->   17
'2'    --->   18
'3'    --->   19
'4'    --->   20
'5'    --->   21
'6'    --->   22
'7'    --->   23
'8'    --->   24
'9'    --->   25
':'    --->   26
';'    --->   27
'<'    --->   28
'='    --->   29
'>'    --->   30
'?'    --->   31
'@'    --->   32
'A'    --->   33
'B'    --->   34
'C'    --->   35
'D'    --->   36
'E'    --->   37
'F'    --->   38
'G'    --->   39
'H'    --->   40
'I'    --->   41
'J'    --->   42
'K'    --->   43
'L'    --->   44
'M'    --->   45
'N'    --->   46
'O'    --->   47
'P'    --->   48
'Q'    --->   49
'R'    --->   50
'S'    --->   51
'T'    --->   52
'U'    --->   53
'V'    --->   54
'W'    --->   55
'X'    --->   56
'Y'    --->   57
'['    --->   

### Conversión de texto a secuencias de enteros

In [4]:
text_as_int = np.array([char2idx[c] for c in text]) # array que asigna un índice a cada caracter del texto

# Mostramos cómo se ha codificado el texto
print ('Texto: ', format(repr(text[:50])), '...'
       '\nCodificado: ', text_as_int[:50])

Texto:  'Prologo\r\nEn 1953, Isaac Asimov publico Segunda Fun' ...
Codificado:  [48 78 75 72 75 67 75  1  0 37 74  2 17 25 21 19 12  2 41 79 61 61 63  2
 33 79 69 73 75 82  2 76 81 62 72 69 63 75  2 51 65 67 81 74 64 61  2 38
 81 74]


### Preparación de los datos de entrenamiento y prueba

In [5]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int) # creamos un dataset con los índices del texto (texto codificado)

seq_length = 100 # longitud de la secuencia de entrada (número de caracteres que se le pasan a la red)

sequences = char_dataset.batch(seq_length + 1, drop_remainder=True) # dividimos el dataset en secuencias de longitud seq_length+1 porque la red predice el siguiente caracter a partir de los seq_length anteriores

In [6]:
# Comprobación de que se ha dividido correctamente
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'Prologo\r\nEn 1953, Isaac Asimov publico Segunda Fundacion, el tercer libro de la saga de la Fundacion '
'(o el decimotercero segun otras fuentes, este es un tema de debate). En Segunda Fundacion aparece por'
' primera vez Arkady Darell, uno de los principales personajes de la parte final de la saga. En su pri'
'mera escena, Arkady, que tiene 14 anos, esta haciendo sus tareas escolares. En concreto, una redaccio'
'n que lleva por titulo ?El Futuro del Plan Sheldon?. Para hacer la redaccion, Arkady esta utilizando '


In [7]:
# Preparación de datos de entrenamiento
# La entrada corresponde al caracter 0 hasta el caracter 99
# La salida corresponde al caracter 1 hasta el caracter 100

def split_input_target(chunk): # función que separa la entrada de la salida
  input_text = chunk[:-1] # entrada
  target_text = chunk[1:] # salida
  return input_text, target_text

dataset = sequences.map(split_input_target) # aplicamos la función anterior a cada secuencia

In [8]:
# Agrupamiento de los datos en lotes de tamaño 64

BATCH_SIZE = 64
BUFFER_SIZE = 10000   # Tamaño del buffer para barajar

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True) # Drop_remainder=True para que todos los lotes tengan el mismo tamaño, los lotes que no se alcancen a completar se descartan
print(dataset)

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>


### Definición del modelo

In [9]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size): # función que crea el modelo
  model = tf.keras.Sequential([
    layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]), # capa de embedding
    layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'), # capa LSTM
    layers.Dense(vocab_size) # capa densa
  ])
  return model

In [10]:
vocab_size = len(vocab) # tamaño del vocabulario
embedding_dim = 256 # dimensión del embedding
rnn_units = 1024 # número de neuronas de la capa LSTM

model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE) # construimos el modelo

model.summary() # mostramos un resumen del modelo

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           23552     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 92)            94300     
                                                                 
Total params: 5364828 (20.47 MB)
Trainable params: 5364828 (20.47 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Entrenamiento del modelo

In [11]:
def loss(labels, logits): # función de pérdida
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [12]:
# Compilación del modelo
model.compile(optimizer='adam', loss=loss)

In [13]:
# Añadir checkpoints para almacenar los pesos del modelo en cada época

checkpoint_dir = './training_checkpoints' # directorio donde se guardarán los checkpoints
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}") # nombre de los archivos de los checkpoints

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
  filepath=checkpoint_prefix,
  save_weights_only=True) # sólo se guardan los pesos

In [15]:
EPOCHS = 50 # número de épocas
model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback]) # entrenamiento del modelo

Epoch 1/50
31/31 [==============================] - 32s 1s/step - loss: 2.8470
Epoch 2/50
31/31 [==============================] - 32s 1s/step - loss: 2.4404
Epoch 3/50
31/31 [==============================] - 31s 998ms/step - loss: 2.2153
Epoch 4/50
31/31 [==============================] - 31s 998ms/step - loss: 2.0782
Epoch 5/50
31/31 [==============================] - 34s 1s/step - loss: 1.9457
Epoch 6/50
31/31 [==============================] - 31s 989ms/step - loss: 1.8158
Epoch 7/50
31/31 [==============================] - 32s 1s/step - loss: 1.6931
Epoch 8/50
31/31 [==============================] - 31s 1s/step - loss: 1.5786
Epoch 9/50
31/31 [==============================] - 31s 996ms/step - loss: 1.4791
Epoch 10/50
31/31 [==============================] - 31s 1s/step - loss: 1.3938
Epoch 11/50
31/31 [==============================] - 32s 1s/step - loss: 1.3119
Epoch 12/50
31/31 [==============================] - 32s 1s/step - loss: 1.2433
Epoch 13/50
31/31 [==================

### Generación de texto

In [16]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1) # construimos el modelo con batch_size=1
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir)) # cargamos los pesos del último checkpoint

model.build(tf.TensorShape([1, None])) # construimos el modelo con batch_size=1, el 1 es porque solo se espera 1 oración de entrada

In [26]:
# Función para generar texto caracter a caracter

def generate_text(model, start_string):
  num_generate = 500 # número de caracteres a generar
  input_eval = [char2idx[s] for s in start_string] # convertimos la cadena de entrada en una lista de índices (números)

  input_eval = tf.expand_dims(input_eval, 0) # añadimos una dimensión al principio (batch_size=1)

  text_generated = [] # lista para almacenar el texto generado

  temperature = 0.5 # parámetro para controlar la aleatoriedad de la predicción (0 = predicción determinista, 1 = predicción aleatoria)

  model.reset_states() # reiniciamos el estado de la red

  for i in range(num_generate):
    predictions = model(input_eval) # predicciones de la red
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature  # dividimos entre temperature para controlar la aleatoriedad de la predicción

    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()  # obtenemos el índice de la predicción (el caracter predicho)
                                                                                    # [-1,0] para obtener el último caracter predicho
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id]) # añadimos el caracter predicho a la lista

  return (start_string + ''.join(text_generated)) # devolvemos la cadena de entrada + la cadena generada

In [29]:
print(generate_text(model, start_string="redes ")) # generamos texto a partir de la cadena "redes "

redes neuronales convolucionales, muy usadas en tareas de vision por computador. 
Las redes neuronales convolucionales avon con el lector, programaremos una red neuronal convolucional nos permite conseguir una entrada adicional fija de x0=1).
En la fase de entrenamiento de un modelo consideremos que tenemos un buen resultado de clasificacion:



Si lo ha probado el lector, en este caso supongo que obtenga el valor de estos parametros examinando el momento de los cientificos y cientificas de dato
